In [1]:
import os
import pandas as pd
os.chdir("D:\insofe\AI\cute4-PRAC")

In [2]:
train=pd.read_csv("train-1546603042473.csv")

In [3]:
test=pd.read_csv("test-1546603743049.csv")

In [4]:
from keras.layers import Dense, SimpleRNN, GRU, LSTM, Embedding # Import layers from Keras
from keras.models import Sequential

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
train.shape

(48699, 2)

In [7]:
test.shape

(8581, 2)

In [8]:
train.head(3)

,categories,converse
0,QUERIES FROM PHARMACY,please to verify instructions for drugname pat...
1,NEW APPOINTMENT,lmovm for patients mother to and schd rov trac...
2,OTHERS,labtype and insurance approval other incoming ...


In [9]:
test.head(3)

,id,converse
0,1,request to speak with rn no given patients ref...
1,2,patients husband ret name spouse for other pat...
2,3,fyi in his szs mom other he has an appointment...


In [13]:
# Print the unique classes and their counts/frequencies
import numpy as np
classes = np.unique(train['categories'], return_counts=True) # np.unique returns a tuple with class names and counts
print(classes[0]) #Print the list of unique classes
print(classes[1]) #Print the list of frequencies of the above classes

['CANCELLATION' 'CHANGE OF HOSPITAL' 'CHANGE OF PHARMACY'
 'CHANGE OF PROVIDER' 'FOLLOW UP ON PREVIOUS REQUEST' 'JUNK' 'LAB RESULTS'
 'MEDICATION RELATED' 'NEW APPOINTMENT' 'OTHERS' 'PRIOR AUTHORIZATION'
 'PROVIDER' 'QUERIES FROM INSURANCE FIRM' 'QUERIES FROM PHARMACY'
 'QUERY ON CURRENT APPOINTMENT' 'REFILL' 'RESCHEDULING'
 'RUNNING LATE TO APPOINTMENT'
 'SHARING OF HEALTH RECORDS (FAX, E-MAIL, ETC.)'
 'SHARING OF LAB RECORDS (FAX, E-MAIL, ETC.)' 'SYMPTOMS']
[ 564  127   47  811  304   18 2253 9010 8907 6253 1043 1677   91 1464
  559 8347 1383  590 3018 1212 1021]


In [14]:
pd.value_counts(train['categories'])

MEDICATION RELATED                               9010
NEW APPOINTMENT                                  8907
REFILL                                           8347
OTHERS                                           6253
SHARING OF HEALTH RECORDS (FAX, E-MAIL, ETC.)    3018
LAB RESULTS                                      2253
PROVIDER                                         1677
QUERIES FROM PHARMACY                            1464
RESCHEDULING                                     1383
SHARING OF LAB RECORDS (FAX, E-MAIL, ETC.)       1212
PRIOR AUTHORIZATION                              1043
SYMPTOMS                                         1021
CHANGE OF PROVIDER                                811
RUNNING LATE TO APPOINTMENT                       590
CANCELLATION                                      564
QUERY ON CURRENT APPOINTMENT                      559
FOLLOW UP ON PREVIOUS REQUEST                     304
CHANGE OF HOSPITAL                                127
QUERIES FROM INSURANCE FIRM 

In [83]:
max_num_words = 100
seq_len = 100
embedding_size = 100

In [84]:
test.converse=test.converse.astype(str)
train.converse=train.converse.astype(str)

In [85]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

train.converse = [token.lower() for token in train.converse]
test.converse = [token.lower() for token in test.converse] 



In [86]:
tokenizer = Tokenizer(num_words=max_num_words) #Tokenizer is used to tokenize text
tokenizer.fit_on_texts(test.converse)


In [87]:
#Check the frequency of vovabulary present in entire corpus= large collection of text
A=tokenizer.word_counts
sorted_by_value = sorted(A.items(), key=lambda kv: kv[1],reverse=True)
print(sorted_by_value[0:100])
print(len(sorted_by_value))


[('to', 27090), ('patients', 14227), ('and', 13323), ('the', 11822), ('for', 11556), ('she', 8904), ('of', 8432), ('patient', 8010), ('timephrase', 7823), ('is', 7227), ('a', 6929), ('mg', 6438), ('drugname', 6293), ('rn', 6229), ('with', 6190), ('on', 6164), ('in', 5650), ('that', 5529), ('her', 5486), ('please', 5255), ('rx', 5133), ('rna', 4849), ('labtype', 4556), ('follow', 4377), ('pm', 4362), ('by', 4318), ('po', 4243), ('appointment', 4219), ('am', 4173), ('mom', 4079), ('at', 4034), ('was', 3952), ('tabs', 3786), ('will', 3692), ('not', 3670), ('he', 3625), ('has', 3608), ('other', 3524), ('from', 3421), ('if', 3418), ('i', 3312), ('this', 3293), ('be', 3217), ('doctor', 3040), ('md', 2897), ('clinical', 2878), ('changes', 2724), ('have', 2711), ('list', 2697), ('x', 2669), ('would', 2605), ('name', 2594), ('message', 2526), ('it', 2459), ('pharmacy', 2387), ('medication', 2372), ('phone', 2361), ('entered', 2309), ('authorized', 2308), ('prescription', 2241), ('states', 2214)

In [88]:
#Check how indexing is given by tokenizer to the vocabulary
A=tokenizer.word_index
sorted_by_value = sorted(A.items(), key=lambda kv: kv[1])
print(sorted_by_value)
print(len(sorted_by_value))


[('to', 1), ('patients', 2), ('and', 3), ('the', 4), ('for', 5), ('she', 6), ('of', 7), ('patient', 8), ('timephrase', 9), ('is', 10), ('a', 11), ('mg', 12), ('drugname', 13), ('rn', 14), ('with', 15), ('on', 16), ('in', 17), ('that', 18), ('her', 19), ('please', 20), ('rx', 21), ('rna', 22), ('labtype', 23), ('follow', 24), ('pm', 25), ('by', 26), ('po', 27), ('appointment', 28), ('am', 29), ('mom', 30), ('at', 31), ('was', 32), ('tabs', 33), ('will', 34), ('not', 35), ('he', 36), ('has', 37), ('other', 38), ('from', 39), ('if', 40), ('i', 41), ('this', 42), ('be', 43), ('doctor', 44), ('md', 45), ('clinical', 46), ('changes', 47), ('have', 48), ('list', 49), ('x', 50), ('would', 51), ('name', 52), ('message', 53), ('it', 54), ('pharmacy', 55), ('medication', 56), ('phone', 57), ('entered', 58), ('authorized', 59), ('prescription', 60), ('states', 61), ('like', 62), ('you', 63), ('can', 64), ('refill', 65), ('advise', 66), ('but', 67), ('then', 68), ('completed', 69), ('new', 70), ('a

In [90]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences




tokenizer = Tokenizer(num_words=max_num_words) #Tokenizer is used to tokenize text
tokenizer.fit_on_texts(train.converse) #Fit this to our corpus

x_train = tokenizer.texts_to_sequences(train.converse) #'text to sequences converts the text to a list of indices
x_train = pad_sequences(x_train, maxlen=50) #pad_sequences makes every sequence a fixed size list by padding with 0s 

x_test = tokenizer.texts_to_sequences(test.converse) 
x_test = pad_sequences(x_test, maxlen=50)

x_train.shape, x_test.shape # Check the dimensions of x_train and x_test  

((48699, 50), (8581, 50))

In [91]:
print(train.converse[1])
print(x_train[1])

lmovm for patients mother to and schd rov tracy dominguez am kellee currie pm rhonda fanning
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  5  2  1  3
 27 25]


In [92]:
unique_labels = list(train.categories.unique())
print(unique_labels)

['QUERIES FROM PHARMACY', 'NEW APPOINTMENT', 'OTHERS', 'MEDICATION RELATED', 'SHARING OF HEALTH RECORDS (FAX, E-MAIL, ETC.)', 'REFILL', 'PRIOR AUTHORIZATION', 'RESCHEDULING', 'SYMPTOMS', 'LAB RESULTS', 'FOLLOW UP ON PREVIOUS REQUEST', 'PROVIDER', 'CHANGE OF PROVIDER', 'SHARING OF LAB RECORDS (FAX, E-MAIL, ETC.)', 'QUERY ON CURRENT APPOINTMENT', 'RUNNING LATE TO APPOINTMENT', 'CANCELLATION', 'CHANGE OF PHARMACY', 'QUERIES FROM INSURANCE FIRM', 'JUNK', 'CHANGE OF HOSPITAL']


In [93]:
from keras.utils import to_categorical # This convers the labels to one-hot vectors(Dummies)

y_train = np.array([unique_labels.index(i) for i in train.categories]) # Convert the word labels to indeces
y_train = to_categorical(y_train) # Dummify the labels
#y_test = np.array([unique_labels.index(i) for i in test.id])
#y_test = to_categorical(y_test)

In [94]:
unique_labels1 = list(test.id.unique())
print(unique_labels1)
y_test = np.array([unique_labels1.index(i) for i in test.id])
y_test = to_categorical(y_test)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

In [1]:
#simple RNN

model = Sequential()
model.add(Embedding(input_dim=20000, input_length=50, output_dim=100))
model.add(SimpleRNN(32, return_sequences=False)) 
model.add(Dense(21, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=20, batch_size = 300,validation_split=0.25)

NameError: name 'Sequential' is not defined

In [99]:
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D


model = Sequential()
model.add(Embedding(20000,
                    100,
                    input_length=50
                    ))
model.add(Conv1D(64, 5, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(21, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()
model.fit(x_train,y_train, epochs=12, batch_size=500,validation_split=0.25)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 50, 100)           2000000   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 46, 64)            32064     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_8 (Dense)              (None, 21)                2709      
Total params: 2,043,093
Trainable params: 2,043,093
Non-trainable params: 0
_________________________________________________________________
Train on 36524 samples, validate on 12175 samples
Epoch 1/12
36524/36524 [==============================] - 29s 801us/step - loss: 2

In [101]:
test_prob = model.predict_classes(x_test)

In [103]:
model = Sequential()
model.add(Embedding(input_dim = 100000, 
                    input_length = 50, 
                    output_dim = 150))
model.add(LSTM(10, return_sequences=True))
model.add(LSTM(10, return_sequences=False))
model.add(Dense(21, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=20, batch_size=500, validation_split=0.25)

Train on 36524 samples, validate on 12175 samples
Epoch 1/20
36524/36524 [==============================] - 62s 2ms/step - loss: 2.7818 - acc: 0.2152 - val_loss: 2.4227 - val_acc: 0.3143
Epoch 2/20
36524/36524 [==============================] - 58s 2ms/step - loss: 2.2278 - acc: 0.3957 - val_loss: 2.0710 - val_acc: 0.4390
Epoch 3/20
36524/36524 [==============================] - 57s 2ms/step - loss: 1.9977 - acc: 0.4388 - val_loss: 1.9275 - val_acc: 0.4518
Epoch 4/20
36524/36524 [==============================] - 57s 2ms/step - loss: 1.9015 - acc: 0.4573 - val_loss: 1.8612 - val_acc: 0.4707
Epoch 5/20
36524/36524 [==============================] - 58s 2ms/step - loss: 1.8479 - acc: 0.4685 - val_loss: 1.8224 - val_acc: 0.4697
Epoch 6/20
36524/36524 [==============================] - 58s 2ms/step - loss: 1.8056 - acc: 0.4735 - val_loss: 1.7900 - val_acc: 0.4674
Epoch 7/20
36524/36524 [==============================] - 58s 2ms/step - loss: 1.7698 - acc: 0.4748 - val_loss: 1.7507 - val_acc